In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

# Data importation

In [2]:
df_init = pd.read_csv('dataset_SCL.csv')

In [3]:
df_init

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,22,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,25,12,2017,Lunes,I,JetSmart SPA,Santiago,Lima
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,27,12,2017,Miercoles,I,JetSmart SPA,Santiago,Lima
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,29,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima


# Data Distribution

All variables are categorical, then value_counts is used in order to know percentages per category

In [12]:
all_vars = df_init.columns.to_list()
all_vars.remove("Fecha-I") #alone is not a feature
all_vars.remove("Fecha-O") #alone is not a feature
all_vars.remove("Vlo-I") #flight identifier, alone it cant be used as feature
all_vars.remove("Vlo-O") #flight identifier, alone it cant be used as feature

for var in all_vars:
    print(f"Variable: {var}")
    print(f"Number of categories: {len(df_init[var].unique())}")
    print(df_init[var].value_counts(normalize = True))
    print("\n")

Variable: Ori-I
Number of categories: 1
SCEL    1.0
Name: Ori-I, dtype: float64


Variable: Des-I
Number of categories: 64
SCFA    0.084846
SPJC    0.077266
SCCF    0.075433
SCTE    0.063880
SCIE    0.058573
          ...   
SBFI    0.000015
SPSO    0.000015
SEQU    0.000015
SEQM    0.000015
SARI    0.000015
Name: Des-I, Length: 64, dtype: float64


Variable: Emp-I
Number of categories: 30
LAN    0.551432
SKU    0.209630
TAM    0.044703
ARG    0.028575
CMP    0.027124
LAW    0.023077
AVA    0.016890
JAT    0.016054
GLO    0.011817
AAL    0.011099
ACA    0.008284
IBE    0.005307
AFR    0.005249
DAL    0.005249
AMX    0.005146
UAL    0.004912
ONE    0.004091
AZA    0.003797
KLM    0.003680
LAP    0.003167
BAW    0.003006
QFU    0.002859
JMR    0.001466
LRC    0.001349
AUT    0.001085
PUE    0.000718
LXP    0.000132
LPE    0.000059
DSM    0.000029
LNE    0.000015
Name: Emp-I, dtype: float64


Variable: Ori-O
Number of categories: 1
SCEL    1.0
Name: Ori-O, dtype: float64


Variable: Des-O

Observations:

1- Ori-I, Ori-O, SIGLAORI are formed by a single category, then it cant be used as a feature

2- Des-I, Des-O, Emp-I, Emp-O, OPERA, SIGLADES are formed by 64, 63, 30, 32, 23, 62 categories respectively, but there are a lot of categories with less than 5% of participation. Using these variables as "raw features" is not a good practice. A solution can be drop rows for  minoritary categories or REGROUP these categories in only one category. For example, Emp-O has only 3 categories (of 32) with more than 5% of participation, so a new variable could be formed using 3 majority categories and another category (regroup 29)  for the rest.

3- AÑO and DIA present the same problem described in 2- . AÑO has only 1 majority category, then it cant be used as feature. DIA does not have categories with at least 5 % of participation 

4- TIPOVUELO, MES, DIANOM are formed by categories with at least 5 % of participation. These variables do not need changes 

# Synthetic features

In [14]:
df_init['Fecha-I'] = pd.to_datetime(df_init['Fecha-I'])
df_init['Fecha-O'] = pd.to_datetime(df_init['Fecha-O'])

# High season
df_init['high_season'] = 0
df_init.loc[(((df_init['Fecha-I'].dt.month == 12) & (df_init['Fecha-I'].dt.day >= 15))
            | ((df_init['Fecha-I'].dt.month == 3) & (df_init['Fecha-I'].dt.day <= 4))
            | (df_init['Fecha-I'].dt.month.isin([1, 2])))
            | (df_init['Fecha-I'].dt.month == 7 & df_init['Fecha-I'].dt.day.between(15, 31))
            | (df_init['Fecha-I'].dt.month == 9 & df_init['Fecha-I'].dt.day.between(11, 30))
            , 'high_season'] = 1

# Minute difference
df_init['min_diff'] = (df_init['Fecha-O'] - df_init['Fecha-I']).dt.total_seconds()/60

# Delay 15 min
df_init['delay_15'] = 0
df_init.loc[(df_init['min_diff']>15), 'delay_15'] = 1

# Period of the day 
df_init['period_day'] = None
df_init.loc[df_init['Fecha-I'].dt.strftime('%H:%M').between('05:00', '11:59'), 'period_day'] = 'morning'
df_init.loc[df_init['Fecha-I'].dt.strftime('%H:%M').between('12:00', '18:59'), 'period_day'] = 'afternoon'
df_init.loc[(df_init['Fecha-I'].dt.strftime('%H:%M').between('19:00', '23:59'))
            | (df_init['Fecha-I'].dt.strftime('%H:%M').between('00:00', '04:59'))
            , 'period_day'] = 'night'

df_init[['high_season', 'min_diff', 'delay_15', 'period_day']].to_csv('synthetic_features.csv') # export to csv